In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Data_Hackathon.csv')

In [3]:
df.head(15)

,Unnamed: 0,Upsell,Increasing instalment,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61
0,NaN,Upgrade,Increasing term,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Sales, by month",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Month,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,...,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23,12/31/23
5,NaN,Volume (units),10000,"10,200","10,404","10,612","10,824","11,041","11,262","11,487",...,"26,916","27,454","28,003","28,563","29,135","29,717","30,312","30,918","31,536","32,167"


In [4]:
df.isnull().sum()

Unnamed: 0               5
Upsell                   3
Increasing instalment    3
Unnamed: 3               4
Unnamed: 4               4
                        ..
Unnamed: 57              4
Unnamed: 58              4
Unnamed: 59              4
Unnamed: 60              4
Unnamed: 61              4
Length: 62, dtype: int64

In [5]:
df.dtypes

Unnamed: 0               object
Upsell                   object
Increasing instalment    object
Unnamed: 3               object
Unnamed: 4               object
                          ...  
Unnamed: 57              object
Unnamed: 58              object
Unnamed: 59              object
Unnamed: 60              object
Unnamed: 61              object
Length: 62, dtype: object

In [6]:
df.describe()

,Unnamed: 0,Upsell,Increasing instalment,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61
count,1,3,3,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
unique,1,3,3,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
top,"Sales, by month",Upgrade,Increasing term,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,...,3/31/23,4/30/23,5/31/23,6/30/23,7/31/23,8/31/23,9/30/23,10/31/23,11/30/23,12/31/23
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


DATA PREPROCESSING

In [9]:
# Drop unnecessary rows
df = df.iloc[4:, :]  # Drop the first 4 rows

# Drop unnecessary columns (if they exist)
columns_to_drop = ['Unnamed: 0', 'Upsell', 'Increasing instalment', 'Unnamed: 3']
existing_columns = [col for col in columns_to_drop if col in df.columns]
df = df.drop(existing_columns, axis=1)

# Rename column headers
num_cols = df.shape[1]  # Get the number of columns in the DataFrame
new_col_names = ['Month'] + [str(month) for month in pd.date_range(start='1/31/2019', end='12/31/2023', freq='M')][:num_cols-1]
df.columns = new_col_names

# Convert to long format
df = pd.melt(df, id_vars=['Month'], var_name='Date', value_name='Volume')

# Handle missing values and non-numeric data
df['Date'] = pd.to_datetime(df['Date'])
df['Volume'] = df['Volume'].replace(',', '', regex=True)
df = df.dropna(subset=['Volume'])

Calculate Eligibility

In [10]:
import pulp

def calculate_eligibility(df):
    # Calculate eligibility for upsells (cash loans)
    df['Upsell_Eligible'] = 0.9 * df['Volume']

    # Calculate eligibility for upgrades (TV upgrades)
    df['Upgrade_Eligible'] = 0
    for i in range(400, len(df)):
        prev_volume = df.iloc[i-400]['Volume']
        prev_upsells = df.iloc[i-400:i]['Upsell'].sum()
        df.loc[i, 'Upgrade_Eligible'] = 0.75 * prev_volume - 0.2 * prev_volume * prev_upsells

    return df
df = calculate_eligibility(df)

ModuleNotFoundError: No module named 'pulp'